In [1]:
import os
import cv2
import h5py
import numpy
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc as spm

from tensorflow import set_random_seed
set_random_seed(42)
numpy.random.seed(42)

amount_crops = 30 # Quantos patches de cada imagem
patch_size = 64 # Tamanho dos patches das amostras
label_size = 64 # Tamanho dos patches das labels
conv_side = 0 # será a borda da convolução?
scale = 2
# BORDER_CUT = 8
BLOCK_STEP = 32
BLOCK_SIZE = 64

In [2]:
def load_data(x_file, y_file):
    data = numpy.array((5,5))
    label = numpy.array((5,5))
    with h5py.File(x_file, 'r') as hfx:
        data = numpy.array(hfx['data'])
    with h5py.File(y_file, 'r') as hfy:
        label = numpy.array(hfy['data'])
    return data, label

In [3]:
def prepare_data(x_path, y_path):

    full_data, full_label = load_data(x_path, y_path)

    fd_shape = full_data.shape
    fl_shape = full_label.shape
    
#     full_data = full_data[fd_shape//2:,:,:]
#     full_label = full_label[fl_shape//2:,:,:]
    
    nums = full_data.shape[0]
    data = numpy.zeros((nums * amount_crops, 1, patch_size, patch_size), dtype=numpy.uint8)
    label = numpy.zeros((nums * amount_crops, 1, label_size, label_size), dtype=numpy.uint8)

    for i in range(full_data.shape[0]):
        
        lr_img = full_data[i, :, :]
        hr_img = full_label[i, :, :]

        shape = hr_img.shape

        lr_img = spm.imresize(lr_img, size=shape, interp='bicubic')

        # produce amount_crops random coordinate to crop training img
        Points_x = numpy.random.randint(0, min(shape[0], shape[1]) - patch_size, amount_crops)
        Points_y = numpy.random.randint(0, min(shape[0], shape[1]) - patch_size, amount_crops)

        for j in range(amount_crops):
            lr_patch = lr_img[Points_x[j]: Points_x[j] + patch_size, Points_y[j]: Points_y[j] + patch_size]
            hr_patch = hr_img[Points_x[j]: Points_x[j] + patch_size, Points_y[j]: Points_y[j] + patch_size]

            lr_patch = lr_patch
            hr_patch = hr_patch

            data[i * amount_crops + j, 0, :, :] = lr_patch
            label[i * amount_crops + j, 0, :, :] = hr_patch
#             label[i * amount_crops + j, 0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]
            
#             import matplotlib
#             matplotlib.use('TKagg')
            
#             plt.imshow(lr_patch)
#             plt.waitforbuttonpress()
#             plt.imshow(hr_patch)
#             plt.waitforbuttonpress()

            # cv2.imshow("lr", lr_patch)
            # cv2.imshow("hr", hr_patch)
            # cv2.waitKey(0)
#             %matplotlib inline
    return data, label


In [4]:
def prepare_crop_data(x_path, y_path):
    
    full_data, full_label = load_data(x_path, y_path)

    data = []
    label = []

    # We are using 8-bit deep 256x256 and 512x512 images for the 
    # subsampled and labels, respectively

    for i in range(full_data.shape[0]):
        
        subsampled_img = full_data[i, :, :]
        label_img = full_label[i, :, :]
        
        shape = label_img.shape

        # just resizing to produce a bicubic interpolated image, since the image is already subsampled
        subsampled_img = spm.imresize(subsampled_img, size=shape, interp='bicubic')

        width_num = (shape[0] - (BLOCK_SIZE - BLOCK_STEP) * 2) // BLOCK_STEP
        height_num = (shape[1] - (BLOCK_SIZE - BLOCK_STEP) * 2) // BLOCK_STEP
        for k in range(width_num):
            for j in range(height_num):
                x = k * BLOCK_STEP
                y = j * BLOCK_STEP
                hr_patch = label_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]
                lr_patch = subsampled_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]
        
                lr = numpy.zeros((1, patch_size, patch_size), dtype=numpy.uint8)
                hr = numpy.zeros((1, label_size, label_size), dtype=numpy.uint8)
                
                lr[0, :, :] = lr_patch
                hr[0, :, :] = hr_patch                
#                 hr[0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]

                data.append(lr)
                label.append(hr)

#                 plt.imshow(lr_patch)
#                 plt.waitforbuttonpress()
#                 plt.imshow(hr_patch)
#                 plt.waitforbuttonpress()

#                 # cv2.imshow("lr", lr_patch)
#                 # cv2.imshow("hr", hr_patch)
#                 # cv2.waitKey(0)
#                 %matplotlib inline
    data = numpy.array(data, dtype=float)
    label = numpy.array(label, dtype=float)
    
    return data, label

In [5]:
def write_hdf5(data, labels, output_filename):
    """
    This function is used to save image data and its label(s) to hdf5 file.
    output_file.h5,contain data and label
    """

    x = data.astype(numpy.uint8)
    y = labels.astype(numpy.uint8)

    with h5py.File(output_filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)
        # h.create_dataset()

In [6]:
def read_training_data(file):
    with h5py.File(file, 'r') as hf:
        data = numpy.array(hf.get('data'))
        label = numpy.array(hf.get('label'))
        train_data = numpy.transpose(data, (0, 2, 3, 1))
        train_label = numpy.transpose(label, (0, 2, 3, 1))
        return train_data, train_label

In [7]:
data, label = prepare_crop_data('sub_data/sub_data_pick.h5', 'ground_data/ground_data_pick.h5')

print(data.shape, label.shape)

write_hdf5(data, label, "64crop_train.h5")

# del(data)
# del(label)


/home/ABTLUS/fernando.furusato/tests/jupyter/lib/python3.5/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


(100352, 1, 64, 64) (100352, 1, 64, 64)


In [8]:
data2, label2 = prepare_crop_data('sub_data/sub_data_thorn.h5', 'ground_data/ground_data_thorn.h5')

print(data2.shape,
label2.shape)

/home/ABTLUS/fernando.furusato/tests/jupyter/lib/python3.5/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


(100352, 1, 64, 64) (100352, 1, 64, 64)


In [11]:
print(data.shape[0] + data2.shape[0])
data3 = numpy.empty((data.shape[0] + data2.shape[0], 1, 64, 64))
label3 = numpy.empty((label.shape[0] + label2.shape[0], 1, 64, 64))

200704


In [12]:
data3[:100352,:,:,:] = data
data3[100352:,:,:,:] = data2

label3[:100352,:,:,:] = label
label3[100352:,:,:,:] = label2

In [13]:
write_hdf5(data3, label3, "64crop_train.h5")

print(data3.shape, label3.shape)

del(data)
del(label)

(200704, 1, 64, 64) (200704, 1, 64, 64)


In [14]:
data, label = prepare_data('sub_data/sub_data_pen.h5', 'ground_data/ground_data_pen.h5')
write_hdf5(data, label, "64test.h5")

print(data.shape, label.shape)

del(data)
del(label)

/home/ABTLUS/fernando.furusato/tests/jupyter/lib/python3.5/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


(15360, 1, 64, 64) (15360, 1, 64, 64)
